#Райфайзенбанк прогнозирование Customer Lifetime Value на 6 месяцев.
Бочаров Алексей(скайп bam271074) 


CRM-системы банков собирают большой объем информации, которая используется для анализа бизнес-процессов. 
В рамках работы над кейсом вы получите часть этих данных и решите одну из самых актуальных для Райффайзенбанка 
задач — предсказание размера будущих доходов от сотрудничества с клиентами. Точный прогноз не только поможет 
сформировать лучшие персонализированные клиентские предложения на рынке, но и позволит повысить качество 
среднесрочного планирования, открывая новые горизонты для развития банка.

Использование данных, появившихся после 2018 года, запрещено.


In [0]:
# подгружаем необходимые библиотеки и фиксируем random state

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
RS = 42


In [0]:
pip list

Package                  Version        
------------------------ ---------------
absl-py                  0.9.0          
alabaster                0.7.12         
albumentations           0.1.12         
altair                   4.0.1          
asgiref                  3.2.3          
astor                    0.8.1          
astropy                  4.0            
atari-py                 0.2.6          
atomicwrites             1.3.0          
attrs                    19.3.0         
audioread                2.1.8          
autograd                 1.3            
Babel                    2.8.0          
backcall                 0.1.0          
backports.tempfile       1.0            
backports.weakref        1.0.post1      
beautifulsoup4           4.6.3          
bleach                   3.1.0          
blis                     0.2.4          
bokeh                    1.4.0          
boto                     2.49.0         
boto3                    1.11.15        
botocore        

In [0]:
#connect to google drive where dataset is
from google.colab import drive
drive.mount ('/content/gdrive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls

gdrive	sample_data


In [0]:
!ls /content/gdrive/'My Drive'/

 b		       'Colab Notebooks'   dataset_FW	 Face_head2_nn10
 books		        dataset_DT	   dataset_PD	 IDAO_2020
'CL_Cup IT 2020_DS_1'   dataset_DTsmall    dataset_PDg


In [0]:
print(os.listdir("../content/gdrive"))

data_root = '../content/gdrive/My Drive/'
print(os.listdir(data_root))

['.shortcut-targets-by-id', 'My Drive', '.Trash']
['b', 'Face_head2_nn10', 'Colab Notebooks', 'IDAO_2020', 'dataset_FW', 'books', 'dataset_DTsmall', 'dataset_DT', 'dataset_PD', 'dataset_PDg', 'CL_Cup IT 2020_DS_1']


Let s make dataframes

In [0]:
seq_path_train =(data_root, "CL_Cup IT 2020_DS_1/CUP_IT_train_data.csv")
train_df = pd.read_csv("/".join(seq_path_train), sep=',', skip_blank_lines=True, header=133)
train_df.head(4_200_000)

,cif_id,dlk_cob_date,gi_smooth_3m,big_city,cu_gender,cu_education_level,cu_empl_area,cu_empl_level,payroll_f,cur_quantity_pl,cur_quantity_mort,cur_quantity_cc,cur_quantity_deposits,cur_quantity_dc,cur_quantity_accounts,cur_quantity_saccounts,cur_quantity_mf,cc_balance,cl_balance,ml_balance,pl_balance,td_volume,ca_volume,sa_volume,mf_volume,dc_cash_spend_v,dc_cash_spend_c,cc_cash_spend_v,cc_cash_spend_c,dc_pos_spend_v,dc_pos_spend_c,cc_pos_spend_v,cc_pos_spend_c,ca_f,rc_session_qnt_cur_mon,cur_qnt_sms,active,standalone_dc_f,standalone_payroll_dc_f,standalone_nonpayroll_dc_f,salary,cu_age,cu_mob,cu_empl_cur_dur_m,is_married,cu_eduaction_level
0,335992,2018-06-30,17.957530,MLN,0,02. Среднее профессиональное,ТОРГОВЛЯ,SUPPORT STAFF,0,1,0,0,0,1,1,0,0,NaN,NaN,NaN,-1700.60184,NaN,4.62528,NaN,NaN,0.0,0,0.0,0,10.41888,3,0.0,0,1,0,1.0,1,0,0,0,1192.8,29.0,144,46.0,1,2
1,335992,2018-04-30,27.720918,MLN,0,02. Среднее профессиональное,ТОРГОВЛЯ,SUPPORT STAFF,0,1,0,0,0,1,1,0,0,NaN,NaN,NaN,-1911.42936,NaN,300.73056,NaN,NaN,168.0,2,0.0,0,190.21536,14,0.0,0,1,0,1.0,1,0,0,0,1192.8,29.0,142,44.0,1,2
2,335992,2018-05-31,21.949626,MLN,0,02. Среднее профессиональное,ТОРГОВЛЯ,SUPPORT STAFF,0,1,0,0,0,1,1,0,0,NaN,NaN,NaN,-1807.09176,NaN,13.04472,NaN,NaN,0.0,0,0.0,0,161.63760,16,0.0,0,1,0,1.0,1,0,0,0,1192.8,29.0,143,45.0,1,2
3,335992,2018-08-31,14.855459,MLN,0,02. Среднее профессиональное,ТОРГОВЛЯ,SUPPORT STAFF,0,1,0,0,0,1,1,0,0,NaN,NaN,NaN,-1484.08752,NaN,0.35184,NaN,NaN,0.0,0,0.0,0,0.00000,0,0.0,0,1,0,1.0,1,0,0,0,1192.8,29.0,146,48.0,1,2
4,335992,2018-03-31,27.917161,MLN,0,02. Среднее профессиональное,ТОРГОВЛЯ,SUPPORT STAFF,0,1,0,0,0,1,1,0,0,NaN,NaN,NaN,-2015.04648,NaN,793.68216,NaN,NaN,333.6,3,0.0,0,322.27032,28,0.0,0,1,0,1.0,1,0,0,0,1192.8,29.0,141,43.0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4161778,120605,2018-06-30,52.379263,MSK,0,02. Среднее профессиональное,ТОРГОВЛЯ,SUPPORT STAFF,0,1,0,0,0,1,1,0,0,NaN,NaN,NaN,-11050.39320,NaN,2.33736,NaN,NaN,33.6,2,0.0,0,0.00000,0,0.0,0,1,5,-100.0,1,1,0,1,960.0,34.0,115,31.0,1,2
4161779,120605,2018-02-28,54.218822,MSK,0,02. Среднее профессиональное,ТОРГОВЛЯ,SUPPORT STAFF,0,1,0,0,0,1,1,0,0,NaN,NaN,NaN,-11773.29480,NaN,2.52936,NaN,NaN,0.0,0,0.0,0,0.00000,0,0.0,0,1,2,-100.0,1,1,0,1,960.0,33.0,111,27.0,1,2
4161780,120605,2018-03-31,53.343644,MSK,0,02. Среднее профессиональное,ТОРГОВЛЯ,SUPPORT STAFF,0,1,0,0,0,1,1,0,0,NaN,NaN,NaN,-11597.15040,NaN,1.29336,NaN,NaN,0.0,0,0.0,0,1.58400,1,0.0,0,1,5,-100.0,1,1,0,1,960.0,33.0,112,28.0,1,2
4161781,120605,2018-04-30,52.248182,MSK,0,02. Среднее профессиональное,ТОРГОВЛЯ,SUPPORT STAFF,0,1,0,0,0,1,1,0,0,NaN,NaN,NaN,-11415.40656,NaN,1.64136,NaN,NaN,0.0,0,0.0,0,0.00000,0,0.0,0,1,2,-100.0,1,1,0,1,960.0,34.0,113,29.0,1,2


Название признака	Описание признака
	
cif_id	Уникальный номер для клиента
	
dlk_cob_date	Временная метка данных в формате YYYY-MM-DD
	
gi_smooth_3m	Доход, который банк получил от данного клиента за данный
	месяц
	
big_city	Город клиента (укрупненный до вариантов Мск/Спб., мил-
	лионник, другой город)
	
cu_gender	Пол клиента
	
cu_education_level	Уровень образования клиента (может быть неактуально)
	
cu_empl_area	Область работы клиента (может быть неактуально)
	
cu_empl_level	Уровень должности клиента (может быть неактуально)
	
payroll_f	Флаг того, что клиент получает з/п на карту банка
	
cur_quantity_pl	Количество персональных кредитов
	
cur_quantity_mort	Количество ипотечных кредитов
	
cur_quantity_cc	Количество кредитных карт
	
cur_quantity_deposits	Количество депозитов
	
cur_quantity_dc	Количество дебетовых карт
	
cur_quantity_accounts	Количество счетов
	
cur_quantity_saccounts	Количество накопительных счетов
	
cur_quantity_mf	Количество инвестиционных продуктов
	
cc_balance	Баланс кредитных карт
	
cl_balance	Баланс автокредитов
	
ml_balance	Баланс ипотеки
	
pl_balance	Баланс кредитов
	
td_volume	Баланс депозитов
	
ca_volume	Баланс счетов


Название признака	Описание признака
	
sa_volume	Баланс накопительных счетов
	
mf_volume	Баланс инвестиций
	
dc_cash_spend_v	Объемы трат с дебетовых карт наличными
	
dc_cash_spend_c	Количество трат с дебетовых карт наличными
	
cc_cash_spend_v	Объемы трат с кредитных карт наличными
	
cc_cash_spend_c	Количество трат с кредитных карт наличными
	
dc_pos_spend_v	Объем POS-транзакций с дебетовых карт
	
dc_pos_spend_c	Количество POS-транзакций с дебетовых карт
	
cc_pos_spend_v	Объем POS-транзакций с кредитных карт
	
cc_pos_spend_c	Количество POS-транзакций с кредитных карт
	
ca_f	Флаг наличия текущего счета
	
rc_session_qnt_cur_mon	Количество сессий в онлайн-банке
	
cur_qnt_sms	Количество полученных СМС
	
active	Является ли клиент активным по определению банка
	
standalone_dc_f	Флаг отдельной дебетовой карты
	
standalone_payroll_dc_f	Флаг отдельной зарплатной карты
	
standalone_nonpayroll_	Флаг отдельной НЕ зарплатной карты
dc_f	
	
salary	Зарплата клиента
	
cu_age	Возраст клиента
	
cu_mob	Сколько клиент уже в банке
	
cu_empl_cur_dur_m	Сколько клиент работает на данной работе (может быть
	неактуально)
	
is_married	Состоит ли клиент в браке


In [0]:
train_df.cif_id.nunique() #number of clients

396576

In [0]:
#let s upload file with test data
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print ('User uploaded file {name} with length {length} bytes'.format(name=fn, \
                                                                       length=len(uploaded[fn])))

Saving CUP_IT_test_data.csv to CUP_IT_test_data.csv


KeyError: ignored

In [0]:
!ls

CUP_IT_test_data.csv  gdrive  sample_data


In [0]:
#let s build df from uploaded file

test_df = pd.read_csv("CUP_IT_test_data.csv", sep=',', skip_blank_lines=True, header=133)
test_df.head(13)

,cif_id,dlk_cob_date,gi_smooth_3m,big_city,cu_gender,cu_education_level,cu_empl_area,cu_empl_level,payroll_f,cur_quantity_pl,cur_quantity_mort,cur_quantity_cc,cur_quantity_deposits,cur_quantity_dc,cur_quantity_accounts,cur_quantity_saccounts,cur_quantity_mf,cc_balance,cl_balance,ml_balance,pl_balance,td_volume,ca_volume,sa_volume,mf_volume,dc_cash_spend_v,dc_cash_spend_c,cc_cash_spend_v,cc_cash_spend_c,dc_pos_spend_v,dc_pos_spend_c,cc_pos_spend_v,cc_pos_spend_c,ca_f,rc_session_qnt_cur_mon,cur_qnt_sms,active,standalone_dc_f,standalone_payroll_dc_f,standalone_nonpayroll_dc_f,salary,cu_age,cu_mob,cu_empl_cur_dur_m,is_married,cu_eduaction_level,rank
0,50514,2018-06-30,17.898438,OTH,1,03. Высшее,ДРУГОЕ,SUPPORT STAFF,0,1,0,0,0,1,2,0,0,NaN,NaN,NaN,-2182.57200,NaN,23.05368,NaN,NaN,0.0,0,0.0,0,0.00000,0,0.0000,0,1,3,-100.0,1,1,0,1,960.000000,48.0,83,63.0,1,3,5
1,50514,2018-02-28,24.355419,OTH,1,03. Высшее,ДРУГОЕ,SUPPORT STAFF,0,1,0,0,0,1,2,0,0,NaN,NaN,NaN,-3210.81288,NaN,3.11064,NaN,NaN,0.0,0,0.0,0,0.00000,0,0.0000,0,1,3,-100.0,1,1,0,1,960.000000,47.0,79,59.0,1,3,1
2,50514,2018-04-30,20.534134,OTH,1,03. Высшее,ДРУГОЕ,SUPPORT STAFF,0,1,0,0,0,1,2,0,0,NaN,NaN,NaN,-2703.79896,NaN,13.08216,NaN,NaN,0.0,0,0.0,0,0.00000,0,0.0000,0,1,4,-100.0,1,1,0,1,960.000000,48.0,81,61.0,1,3,3
3,50514,2018-01-31,26.477938,OTH,1,03. Высшее,ДРУГОЕ,SUPPORT STAFF,0,1,0,0,0,1,2,0,0,NaN,NaN,NaN,-3462.84264,NaN,19.72488,NaN,NaN,0.0,0,0.0,0,0.00000,0,0.0000,0,1,2,-100.0,1,1,0,1,960.000000,47.0,78,58.0,1,3,0
4,50514,2018-05-31,19.623962,OTH,1,03. Высшее,ДРУГОЕ,SUPPORT STAFF,0,1,0,0,0,1,2,0,0,NaN,NaN,NaN,-2445.49896,NaN,30.06792,NaN,NaN,0.0,0,0.0,0,0.00000,0,0.0000,0,1,1,-100.0,1,1,0,1,960.000000,48.0,82,62.0,1,3,4
5,50514,2018-03-31,22.532602,OTH,1,03. Высшее,ДРУГОЕ,SUPPORT STAFF,0,1,0,0,0,1,2,0,0,NaN,NaN,NaN,-2959.63704,NaN,20.09640,NaN,NaN,0.0,0,0.0,0,0.00000,0,0.0000,0,1,0,-100.0,1,1,0,1,960.000000,47.0,80,60.0,1,3,2
6,195814,2018-03-31,34.897970,OTH,0,02. Среднее профессиональное,"Строительство, недвижимость",SALES MANAGER,0,1,0,2,0,1,1,0,0,-975.35472,NaN,NaN,-3241.08024,NaN,3.60480,NaN,NaN,0.0,0,0.0,0,0.00000,0,2.5200,1,1,0,1.0,1,1,0,1,768.000000,39.0,82,26.0,0,2,2
7,195814,2018-01-31,35.065316,OTH,0,02. Среднее профессиональное,"Строительство, недвижимость",SALES MANAGER,0,1,0,2,0,1,1,0,0,-161.36808,NaN,NaN,-3695.90976,NaN,3.01536,NaN,NaN,0.0,0,0.0,0,0.00000,0,0.0000,0,1,0,1.0,1,1,0,1,768.000000,39.0,80,24.0,0,2,0
8,195814,2018-06-30,17.881938,OTH,0,02. Среднее профессиональное,"Строительство, недвижимость",SALES MANAGER,0,0,0,1,0,0,1,0,0,0.00000,NaN,NaN,NaN,NaN,0.00984,NaN,NaN,0.0,0,0.0,0,0.00000,0,2.3760,2,1,0,-100.0,1,0,0,0,1279.200000,39.0,85,29.0,0,2,5
9,195814,2018-04-30,37.498011,OTH,0,02. Среднее профессиональное,"Строительство, недвижимость",SALES MANAGER,0,1,0,2,0,1,1,0,0,-966.62472,NaN,NaN,-3010.22976,NaN,3.29952,NaN,NaN,0.0,0,0.0,0,0.00000,0,116.9808,9,1,0,1.0,1,1,0,1,768.000000,39.0,83,27.0,0,2,3


In [0]:
#let s build df from the file in googledrive

#seq_path_test =(data_root, "CL_Cup IT 2020_DS_1/CUP_IT_test_data.csv")
#test_df = pd.read_csv("/".join(seq_path_test), sep=',', skip_blank_lines=True, header=133)
#test_df.head(13)


rank - колонку не используем. ее добавили по ошибке.

In [0]:
X_colomns = ['cif_id',	'dlk_cob_date',	'gi_smooth_3m', 'cu_age'] #take in work minimum columns

X_work = train_df[X_colomns]
X_work.head(20)

,cif_id,dlk_cob_date,gi_smooth_3m,cu_age
0,335992,2018-06-30,17.957530,29.0
1,335992,2018-04-30,27.720918,29.0
2,335992,2018-05-31,21.949626,29.0
3,335992,2018-08-31,14.855459,29.0
4,335992,2018-03-31,27.917161,29.0
5,335992,2018-10-31,12.874664,30.0
6,335992,2018-01-31,24.804408,29.0
7,335992,2018-11-30,11.729811,30.0
8,335992,2018-09-30,13.859950,30.0
9,335992,2018-02-28,26.167117,29.0


In [0]:
X_work.info() #let s see columns type

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4161783 entries, 0 to 4161782
Data columns (total 4 columns):
cif_id          int64
dlk_cob_date    object
gi_smooth_3m    float64
cu_age          float64
dtypes: float64(2), int64(1), object(1)
memory usage: 127.0+ MB


In [0]:
X_work.dlk_cob_date =  pd.to_datetime(X_work.dlk_cob_date) # convert object to date
X_work.head(45)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,cif_id,dlk_cob_date,gi_smooth_3m,cu_age
0,335992,2018-06-30,17.957530,29.0
1,335992,2018-04-30,27.720918,29.0
2,335992,2018-05-31,21.949626,29.0
3,335992,2018-08-31,14.855459,29.0
4,335992,2018-03-31,27.917161,29.0
5,335992,2018-10-31,12.874664,30.0
6,335992,2018-01-31,24.804408,29.0
7,335992,2018-11-30,11.729811,30.0
8,335992,2018-09-30,13.859950,30.0
9,335992,2018-02-28,26.167117,29.0


In [0]:
X_work.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4161783 entries, 0 to 4161782
Data columns (total 4 columns):
cif_id          int64
dlk_cob_date    datetime64[ns]
gi_smooth_3m    float64
cu_age          float64
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 127.0 MB


In [0]:
#lets make new column month
X_work['month'] =  X_work['dlk_cob_date'].map(lambda x: 'month_'+str(x.month))
X_work.head(45)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,cif_id,dlk_cob_date,gi_smooth_3m,cu_age,month
0,335992,2018-06-30,17.957530,29.0,month_6
1,335992,2018-04-30,27.720918,29.0,month_4
2,335992,2018-05-31,21.949626,29.0,month_5
3,335992,2018-08-31,14.855459,29.0,month_8
4,335992,2018-03-31,27.917161,29.0,month_3
5,335992,2018-10-31,12.874664,30.0,month_10
6,335992,2018-01-31,24.804408,29.0,month_1
7,335992,2018-11-30,11.729811,30.0,month_11
8,335992,2018-09-30,13.859950,30.0,month_9
9,335992,2018-02-28,26.167117,29.0,month_2


In [0]:
#lets sort by columns client, month
X_work =  X_work.sort_values(by =['cif_id', 'month'], ascending=[True, True])
X_work.head(4_200_000)

,cif_id,dlk_cob_date,gi_smooth_3m,cu_age,month
1342694,1001,2018-01-31,70.994186,56.0,month_1
1342700,1001,2018-10-31,134.287180,57.0,month_10
1342697,1001,2018-11-30,10.396075,57.0,month_11
1342695,1001,2018-12-31,11.512196,57.0,month_12
1342693,1001,2018-02-28,-6.005108,56.0,month_2
...,...,...,...,...,...
1121531,511537,2018-05-31,43.237258,31.0,month_5
1121528,511537,2018-06-30,42.563590,31.0,month_6
1121532,511537,2018-07-31,45.334837,31.0,month_7
1121527,511537,2018-08-31,44.836783,31.0,month_8


In [0]:
X_work.describe()

,cif_id,gi_smooth_3m,cu_age
count,4.161783e+06,4.161783e+06,4.161783e+06
mean,2.561075e+05,4.017062e+01,3.999270e+01
std,1.473086e+05,5.402052e+01,9.437946e+00
min,1.001000e+03,-1.199554e+02,-1.000000e+02
25%,1.285530e+05,4.179842e+00,3.200000e+01
50%,2.559730e+05,2.197043e+01,3.800000e+01
75%,3.837020e+05,5.714864e+01,4.700000e+01
max,5.115370e+05,4.723268e+03,9.000000e+01


In [0]:
X_work.count(axis=0) #let s see if where is NaN in data

cif_id          4161783
dlk_cob_date    4161783
gi_smooth_3m    4161783
cu_age          4161783
month           4161783
dtype: int64

In [0]:
#let s make one hot encoding
#X_work.types.astype(str).str.get_dummies()

df_nonbinary = pd.get_dummies(X_work['month'])
df_nonbinary.head(13)

,month_1,month_10,month_11,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9
1342694,1,0,0,0,0,0,0,0,0,0,0,0
1342700,0,1,0,0,0,0,0,0,0,0,0,0
1342697,0,0,1,0,0,0,0,0,0,0,0,0
1342695,0,0,0,1,0,0,0,0,0,0,0,0
1342693,0,0,0,0,1,0,0,0,0,0,0,0
1342691,0,0,0,0,0,1,0,0,0,0,0,0
1342690,0,0,0,0,0,0,1,0,0,0,0,0
1342692,0,0,0,0,0,0,0,1,0,0,0,0
1342698,0,0,0,0,0,0,0,0,1,0,0,0
1342696,0,0,0,0,0,0,0,0,0,1,0,0


In [0]:
#let s concatenate X_work and df_nonbinary

#Xy= pd.merge(X_work, df_nonbinary)
Xy = pd.concat([X_work, df_nonbinary], axis=1)
Xy.head(13)

,cif_id,dlk_cob_date,gi_smooth_3m,cu_age,month,month_1,month_10,month_11,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9
1342694,1001,2018-01-31,70.994186,56.0,month_1,1,0,0,0,0,0,0,0,0,0,0,0
1342700,1001,2018-10-31,134.287180,57.0,month_10,0,1,0,0,0,0,0,0,0,0,0,0
1342697,1001,2018-11-30,10.396075,57.0,month_11,0,0,1,0,0,0,0,0,0,0,0,0
1342695,1001,2018-12-31,11.512196,57.0,month_12,0,0,0,1,0,0,0,0,0,0,0,0
1342693,1001,2018-02-28,-6.005108,56.0,month_2,0,0,0,0,1,0,0,0,0,0,0,0
1342691,1001,2018-03-31,114.989605,56.0,month_3,0,0,0,0,0,1,0,0,0,0,0,0
1342690,1001,2018-04-30,147.408528,56.0,month_4,0,0,0,0,0,0,1,0,0,0,0,0
1342692,1001,2018-05-31,185.987934,56.0,month_5,0,0,0,0,0,0,0,1,0,0,0,0
1342698,1001,2018-06-30,44.345863,56.0,month_6,0,0,0,0,0,0,0,0,1,0,0,0
1342696,1001,2018-07-31,-17.141319,56.0,month_7,0,0,0,0,0,0,0,0,0,1,0,0


In [0]:
Xy.columns

Index(['cif_id', 'dlk_cob_date', 'gi_smooth_3m', 'cu_age', 'month', 'month_1',
       'month_10', 'month_11', 'month_12', 'month_2', 'month_3', 'month_4',
       'month_5', 'month_6', 'month_7', 'month_8', 'month_9'],
      dtype='object')

In [0]:
lst = []
for n, s in zip (Xy.gi_smooth_3m, Xy.month_1): #make new column Jan with data
  lst.append(n*s) 
Xy['Jan'] = lst

Xy.head()

,cif_id,dlk_cob_date,gi_smooth_3m,cu_age,month,month_1,month_10,month_11,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,Jan
1342694,1001,2018-01-31,70.994186,56.0,month_1,1,0,0,0,0,0,0,0,0,0,0,0,70.994186
1342700,1001,2018-10-31,134.287180,57.0,month_10,0,1,0,0,0,0,0,0,0,0,0,0,0.000000
1342697,1001,2018-11-30,10.396075,57.0,month_11,0,0,1,0,0,0,0,0,0,0,0,0,0.000000
1342695,1001,2018-12-31,11.512196,57.0,month_12,0,0,0,1,0,0,0,0,0,0,0,0,0.000000
1342693,1001,2018-02-28,-6.005108,56.0,month_2,0,0,0,0,1,0,0,0,0,0,0,0,-0.000000


In [0]:
lst = []
for n, s in zip (Xy.gi_smooth_3m, Xy.month_2): #make new column Feb with data
  lst.append(n*s) 
Xy['Feb'] = lst

In [0]:
lst = []
for n, s in zip (Xy.gi_smooth_3m, Xy.month_3): #make new column March with data
  lst.append(n*s) 
Xy['March'] = lst

Xy.head()

,cif_id,dlk_cob_date,gi_smooth_3m,cu_age,month,month_1,month_10,month_11,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,Jan,Feb,March
1342694,1001,2018-01-31,70.994186,56.0,month_1,1,0,0,0,0,0,0,0,0,0,0,0,70.994186,0.000000,0.0
1342700,1001,2018-10-31,134.287180,57.0,month_10,0,1,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.0
1342697,1001,2018-11-30,10.396075,57.0,month_11,0,0,1,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.0
1342695,1001,2018-12-31,11.512196,57.0,month_12,0,0,0,1,0,0,0,0,0,0,0,0,0.000000,0.000000,0.0
1342693,1001,2018-02-28,-6.005108,56.0,month_2,0,0,0,0,1,0,0,0,0,0,0,0,-0.000000,-6.005108,-0.0


In [0]:
lst = []
for n, s in zip (Xy.gi_smooth_3m, Xy.month_4): #make new column Apr with data
  lst.append(n*s) 
Xy['Apr'] = lst

In [0]:
lst = []
for n, s in zip (Xy.gi_smooth_3m, Xy.month_5): #make new column May with data
  lst.append(n*s) 
Xy['May'] = lst

In [0]:
lst = []
for n, s in zip (Xy.gi_smooth_3m, Xy.month_6): #make new column June with data
  lst.append(n*s) 
Xy['June'] = lst

In [0]:
lst = []
for n, s in zip (Xy.gi_smooth_3m, Xy.month_7): #make new column July with data
  lst.append(n*s) 
Xy['July'] = lst

In [0]:
lst = []
for n, s in zip (Xy.gi_smooth_3m, Xy.month_8): #make new column August with data
  lst.append(n*s) 
Xy['Aug'] = lst

In [0]:
lst = []
for n, s in zip (Xy.gi_smooth_3m, Xy.month_9): #make new column September with data
  lst.append(n*s) 
Xy['Sep'] = lst

In [0]:
lst = []
for n, s in zip (Xy.gi_smooth_3m, Xy.month_10): #make new column Oct with data
  lst.append(n*s) 
Xy['Oct'] = lst

In [0]:
lst = []
for n, s in zip (Xy.gi_smooth_3m, Xy.month_11): #make new column November with data
  lst.append(n*s) 
Xy['Nov'] = lst

In [0]:
lst = []
for n, s in zip (Xy.gi_smooth_3m, Xy.month_12): #make new column December with data
  lst.append(n*s) 
Xy['Dec'] = lst
Xy.head()

,cif_id,dlk_cob_date,gi_smooth_3m,cu_age,month,month_1,month_10,month_11,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,Jan,Feb,March,Apr,May,June,July,Aug,Sep,Oct,Nov,Dec
1342694,1001,2018-01-31,70.994186,56.0,month_1,1,0,0,0,0,0,0,0,0,0,0,0,70.994186,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000
1342700,1001,2018-10-31,134.287180,57.0,month_10,0,1,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,134.28718,0.000000,0.000000
1342697,1001,2018-11-30,10.396075,57.0,month_11,0,0,1,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,10.396075,0.000000
1342695,1001,2018-12-31,11.512196,57.0,month_12,0,0,0,1,0,0,0,0,0,0,0,0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,11.512196
1342693,1001,2018-02-28,-6.005108,56.0,month_2,0,0,0,0,1,0,0,0,0,0,0,0,-0.000000,-6.005108,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.00000,-0.000000,-0.000000


In [0]:
Xy.shape

(4161783, 29)

In [0]:
def my_sum(x_df):
  dfout = sum (x_df['Jan'])
  return dfout
Xy_Jan = Xy.groupby(['cif_id']).apply(my_sum)
Xy_Jan.head(13)

cif_id
1001     70.994186
1002     48.866904
1004     -0.010723
1005      0.008142
1007      1.007295
1009     26.129680
1010    119.592187
1011     62.908057
1013      0.813200
1014     24.451243
1016     10.516848
1018      8.986404
1019      4.058193
dtype: float64

In [0]:
Xy_Jan.shape

(396576,)

In [0]:
def my_sum(x_df):
  dfout = sum (x_df['Feb'])
  return dfout
Xy_Feb = Xy.groupby(['cif_id']).apply(my_sum)
Xy_Feb.shape


(396576,)

In [0]:
def my_sum(x_df):
  dfout = sum (x_df['March'])
  return dfout
Xy_March = Xy.groupby(['cif_id']).apply(my_sum)
Xy_March.shape


(396576,)

In [0]:
def my_sum(x_df):
  dfout = sum (x_df['Apr'])
  return dfout
Xy_Apr = Xy.groupby(['cif_id']).apply(my_sum)
Xy_Apr.shape


(396576,)

In [0]:
def my_sum(x_df):
  dfout = sum (x_df['May'])
  return dfout
Xy_May = Xy.groupby(['cif_id']).apply(my_sum)
Xy_May.shape


(396576,)

In [0]:
def my_sum(x_df):
  dfout = sum (x_df['June'])
  return dfout
Xy_June = Xy.groupby(['cif_id']).apply(my_sum)
Xy_June.shape


(396576,)

In [0]:
def my_sum(x_df):
  dfout = sum (x_df['July'])
  return dfout
Xy_July = Xy.groupby(['cif_id']).apply(my_sum)
Xy_July.shape


(396576,)

In [0]:
def my_sum(x_df):
  dfout = sum (x_df['Aug'])
  return dfout
Xy_Aug = Xy.groupby(['cif_id']).apply(my_sum)
Xy_Aug.shape


(396576,)

In [0]:
def my_sum(x_df):
  dfout = sum (x_df['Sep'])
  return dfout
Xy_Sep = Xy.groupby(['cif_id']).apply(my_sum)
Xy_Sep.shape


(396576,)

In [0]:
def my_sum(x_df):
  dfout = sum (x_df['Oct'])
  return dfout
Xy_Oct = Xy.groupby(['cif_id']).apply(my_sum)
Xy_Oct.shape


(396576,)

In [0]:
def my_sum(x_df):
  dfout = sum (x_df['Nov'])
  return dfout
Xy_Nov = Xy.groupby(['cif_id']).apply(my_sum)
Xy_Nov.shape


(396576,)

In [0]:
def my_sum(x_df):
  dfout = sum (x_df['Dec'])
  return dfout
Xy_Dec = Xy.groupby(['cif_id']).apply(my_sum)
Xy_Dec.shape


(396576,)

In [0]:
X = pd.concat([Xy_Jan, Xy_Feb, Xy_March, Xy_Apr, Xy_May, Xy_June], axis=1)
X.head(13)

,0,1,2,3,4,5
cif_id,,,,,,
1001,70.994186,-6.005108,114.989605,147.408528,185.987934,44.345863
1002,48.866904,44.707869,41.433222,36.743185,31.648160,27.042967
1004,-0.010723,0.000313,0.000321,0.000317,0.831670,6.434866
1005,0.008142,0.007914,0.007829,0.007742,0.008036,0.007969
1007,1.007295,0.990931,1.461097,2.130258,2.629876,2.323935
1009,26.129680,23.062523,18.751658,18.059010,27.013554,29.271032
1010,119.592187,118.844432,113.083664,104.606436,74.321493,34.985108
1011,62.908057,70.049570,76.723632,168.230816,216.706839,252.091164
1013,0.813200,0.790338,0.781774,0.773252,0.802499,0.795807


In [0]:
y = pd.concat([Xy_July, Xy_Aug, Xy_Sep, Xy_Oct, Xy_Nov, Xy_Dec], axis=1)
y.head(13)

,0,1,2,3,4,5
cif_id,,,,,,
1001,-17.141319,97.053514,86.175717,134.287180,10.396075,11.512196
1002,23.329385,22.858042,22.249386,21.738326,20.898018,20.740152
1004,14.812981,15.427822,36.044740,31.132410,29.671605,3.453630
1005,0.008022,0.007987,0.007984,0.007979,0.007890,0.007982
1007,1.657362,1.146276,0.536943,0.534138,1.041207,1.734690
1009,29.243226,21.197539,21.667405,22.642134,22.437222,19.202729
1010,5.606455,2.421402,3.226507,1.628539,0.814824,0.012001
1011,422.076430,428.712547,403.448890,182.418386,163.047079,180.472945
1013,0.801028,0.797537,0.797214,0.796683,0.787861,0.797062


In [0]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

#сделаем обучающую выборку

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = RS, shuffle=False)

In [0]:
net = MLPRegressor(hidden_layer_sizes=(6, 12, 6),verbose=False, \
                   early_stopping=True, shuffle=False, random_state=RS)

net.fit(X_train,y_train)

#prediction = net.predict(X_test)[0]
prediction = net.predict(X_test)
print (prediction)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multi

[[197.38776971 201.51222476 197.5484907  190.00517362 183.12467382
  175.32689753]
 [112.13403249 123.22807643 112.57064787 104.49306161 103.56332884
   98.04340913]
 [  1.04175391   1.95143717   2.33220307   3.09872376   4.08031885
    4.59082869]
 ...
 [ 16.96584399  19.39956785  20.64151217  20.20748146  20.54956423
   20.03561523]
 [  8.25961236   9.42727617  10.02303209  10.62778964  11.25897082
   11.49492223]
 [ 43.4212164   43.81624886  43.55162437  42.63293463  42.06142767
   40.89503118]]


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


In [0]:
def smape(predicted_values, true_values): 
    # metric
    return np.mean(np.abs((predicted_values - true_values) 
        / (np.abs(predicted_values) + np.abs(true_values))))

In [0]:
smape(prediction, y_test)

0    0.325641
1    0.395516
2    0.396002
3    0.422364
4    0.460019
5    0.460050
dtype: float64

In [0]:
print(prediction)

[[197.38776971 201.51222476 197.5484907  190.00517362 183.12467382
  175.32689753]
 [112.13403249 123.22807643 112.57064787 104.49306161 103.56332884
   98.04340913]
 [  1.04175391   1.95143717   2.33220307   3.09872376   4.08031885
    4.59082869]
 ...
 [ 16.96584399  19.39956785  20.64151217  20.20748146  20.54956423
   20.03561523]
 [  8.25961236   9.42727617  10.02303209  10.62778964  11.25897082
   11.49492223]
 [ 43.4212164   43.81624886  43.55162437  42.63293463  42.06142767
   40.89503118]]


In [0]:
pred = pd.DataFrame(prediction)

In [0]:
pred.head(13)

,0,1,2,3,4,5
0,197.387770,201.512225,197.548491,190.005174,183.124674,175.326898
1,112.134032,123.228076,112.570648,104.493062,103.563329,98.043409
2,1.041754,1.951437,2.332203,3.098724,4.080319,4.590829
3,6.974810,8.341455,9.179657,9.761888,10.424491,10.656304
4,27.967218,37.208425,34.284421,31.461297,32.166323,30.376175
5,3.632512,9.326784,15.630399,18.685836,19.932419,21.058753
6,264.933518,262.972529,260.498293,251.769851,242.009431,232.247107
7,144.531466,137.816346,140.254719,137.107428,132.404291,128.022141
8,3.632512,9.326784,15.630399,18.685836,19.932419,21.058753
9,32.703047,34.130086,33.567458,33.004086,32.851145,32.065930


In [0]:
pred.shape

(118973, 6)

In [0]:
#let s form the final file

X_colomns = ['cif_id',	'dlk_cob_date',	'gi_smooth_3m', 'cu_age'] #take in work minimum columns

y_test = test_df[X_colomns]
y_test.head(20)

,cif_id,dlk_cob_date,gi_smooth_3m,cu_age
0,50514,2018-06-30,17.898438,48.0
1,50514,2018-02-28,24.355419,47.0
2,50514,2018-04-30,20.534134,48.0
3,50514,2018-01-31,26.477938,47.0
4,50514,2018-05-31,19.623962,48.0
5,50514,2018-03-31,22.532602,47.0
6,195814,2018-03-31,34.897970,39.0
7,195814,2018-01-31,35.065316,39.0
8,195814,2018-06-30,17.881938,39.0
9,195814,2018-04-30,37.498011,39.0


In [0]:
y_test.dlk_cob_date =  pd.to_datetime(y_test.dlk_cob_date) # convert object to date
y_test.head(45)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,cif_id,dlk_cob_date,gi_smooth_3m,cu_age
0,50514,2018-06-30,17.898438,48.0
1,50514,2018-02-28,24.355419,47.0
2,50514,2018-04-30,20.534134,48.0
3,50514,2018-01-31,26.477938,47.0
4,50514,2018-05-31,19.623962,48.0
5,50514,2018-03-31,22.532602,47.0
6,195814,2018-03-31,34.897970,39.0
7,195814,2018-01-31,35.065316,39.0
8,195814,2018-06-30,17.881938,39.0
9,195814,2018-04-30,37.498011,39.0


In [0]:
y_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683772 entries, 0 to 683771
Data columns (total 4 columns):
cif_id          683772 non-null int64
dlk_cob_date    683772 non-null datetime64[ns]
gi_smooth_3m    683772 non-null float64
cu_age          683772 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 20.9 MB


In [0]:
#lets make new column month
y_test['month'] =  y_test['dlk_cob_date'].map(lambda x: 'month_'+str(x.month))
y_test.head(45)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,cif_id,dlk_cob_date,gi_smooth_3m,cu_age,month
0,50514,2018-06-30,17.898438,48.0,month_6
1,50514,2018-02-28,24.355419,47.0,month_2
2,50514,2018-04-30,20.534134,48.0,month_4
3,50514,2018-01-31,26.477938,47.0,month_1
4,50514,2018-05-31,19.623962,48.0,month_5
5,50514,2018-03-31,22.532602,47.0,month_3
6,195814,2018-03-31,34.897970,39.0,month_3
7,195814,2018-01-31,35.065316,39.0,month_1
8,195814,2018-06-30,17.881938,39.0,month_6
9,195814,2018-04-30,37.498011,39.0,month_4


In [0]:
#lets sort by columns client, month
y_test =  y_test.sort_values(by =['cif_id', 'month'], ascending=[True, True])
y_test.head(37)

,cif_id,dlk_cob_date,gi_smooth_3m,cu_age,month
254464,1000,2018-01-31,5.318406e+01,32.0,month_1
254461,1000,2018-02-28,4.759214e+01,33.0,month_2
254460,1000,2018-03-31,4.082896e+01,33.0,month_3
254462,1000,2018-04-30,3.596717e+01,33.0,month_4
254465,1000,2018-05-31,6.179442e+01,33.0,month_5
254463,1000,2018-06-30,8.427602e+01,33.0,month_6
526717,1003,2018-01-31,6.905228e+01,34.0,month_1
526718,1003,2018-02-28,7.079595e+01,34.0,month_2
526716,1003,2018-03-31,6.320246e+01,35.0,month_3
526721,1003,2018-04-30,8.353512e+01,35.0,month_4


In [0]:
df_nonbinary = pd.get_dummies(y_test['month'])
df_nonbinary.head(13)

,month_1,month_10,month_11,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9
254464,1,0,0,0,0,0,0,0,0,0,0
254461,0,0,0,1,0,0,0,0,0,0,0
254460,0,0,0,0,1,0,0,0,0,0,0
254462,0,0,0,0,0,1,0,0,0,0,0
254465,0,0,0,0,0,0,1,0,0,0,0
254463,0,0,0,0,0,0,0,1,0,0,0
526717,1,0,0,0,0,0,0,0,0,0,0
526718,0,0,0,1,0,0,0,0,0,0,0
526716,0,0,0,0,1,0,0,0,0,0,0
526721,0,0,0,0,0,1,0,0,0,0,0


In [0]:
#let s concatenate df

y = pd.concat([y_test, df_nonbinary], axis=1)
y.head(13)

,cif_id,dlk_cob_date,gi_smooth_3m,cu_age,month,month_1,month_10,month_11,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9
254464,1000,2018-01-31,53.184058,32.0,month_1,1,0,0,0,0,0,0,0,0,0,0
254461,1000,2018-02-28,47.592137,33.0,month_2,0,0,0,1,0,0,0,0,0,0,0
254460,1000,2018-03-31,40.828960,33.0,month_3,0,0,0,0,1,0,0,0,0,0,0
254462,1000,2018-04-30,35.967168,33.0,month_4,0,0,0,0,0,1,0,0,0,0,0
254465,1000,2018-05-31,61.794418,33.0,month_5,0,0,0,0,0,0,1,0,0,0,0
254463,1000,2018-06-30,84.276016,33.0,month_6,0,0,0,0,0,0,0,1,0,0,0
526717,1003,2018-01-31,69.052279,34.0,month_1,1,0,0,0,0,0,0,0,0,0,0
526718,1003,2018-02-28,70.795953,34.0,month_2,0,0,0,1,0,0,0,0,0,0,0
526716,1003,2018-03-31,63.202465,35.0,month_3,0,0,0,0,1,0,0,0,0,0,0
526721,1003,2018-04-30,83.535120,35.0,month_4,0,0,0,0,0,1,0,0,0,0,0


In [0]:
lst = []
for n, s in zip (y.gi_smooth_3m, y.month_1): #make new column Jan with data
  lst.append(n*s) 
y['Jan'] = lst

y.head()

,cif_id,dlk_cob_date,gi_smooth_3m,cu_age,month,month_1,month_10,month_11,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,Jan
254464,1000,2018-01-31,53.184058,32.0,month_1,1,0,0,0,0,0,0,0,0,0,0,53.184058
254461,1000,2018-02-28,47.592137,33.0,month_2,0,0,0,1,0,0,0,0,0,0,0,0.000000
254460,1000,2018-03-31,40.828960,33.0,month_3,0,0,0,0,1,0,0,0,0,0,0,0.000000
254462,1000,2018-04-30,35.967168,33.0,month_4,0,0,0,0,0,1,0,0,0,0,0,0.000000
254465,1000,2018-05-31,61.794418,33.0,month_5,0,0,0,0,0,0,1,0,0,0,0,0.000000


In [0]:
lst = []
for n, s in zip (y.gi_smooth_3m, y.month_2): #make new column Feb with data
  lst.append(n*s) 
y['Feb'] = lst

y.head()

,cif_id,dlk_cob_date,gi_smooth_3m,cu_age,month,month_1,month_10,month_11,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,Jan,Feb
254464,1000,2018-01-31,53.184058,32.0,month_1,1,0,0,0,0,0,0,0,0,0,0,53.184058,0.000000
254461,1000,2018-02-28,47.592137,33.0,month_2,0,0,0,1,0,0,0,0,0,0,0,0.000000,47.592137
254460,1000,2018-03-31,40.828960,33.0,month_3,0,0,0,0,1,0,0,0,0,0,0,0.000000,0.000000
254462,1000,2018-04-30,35.967168,33.0,month_4,0,0,0,0,0,1,0,0,0,0,0,0.000000,0.000000
254465,1000,2018-05-31,61.794418,33.0,month_5,0,0,0,0,0,0,1,0,0,0,0,0.000000,0.000000


In [0]:
lst = []
for n, s in zip (y.gi_smooth_3m, y.month_3): #make new column March with data
  lst.append(n*s) 
y['March'] = lst

y.head()

,cif_id,dlk_cob_date,gi_smooth_3m,cu_age,month,month_1,month_10,month_11,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,Jan,Feb,March
254464,1000,2018-01-31,53.184058,32.0,month_1,1,0,0,0,0,0,0,0,0,0,0,53.184058,0.000000,0.00000
254461,1000,2018-02-28,47.592137,33.0,month_2,0,0,0,1,0,0,0,0,0,0,0,0.000000,47.592137,0.00000
254460,1000,2018-03-31,40.828960,33.0,month_3,0,0,0,0,1,0,0,0,0,0,0,0.000000,0.000000,40.82896
254462,1000,2018-04-30,35.967168,33.0,month_4,0,0,0,0,0,1,0,0,0,0,0,0.000000,0.000000,0.00000
254465,1000,2018-05-31,61.794418,33.0,month_5,0,0,0,0,0,0,1,0,0,0,0,0.000000,0.000000,0.00000


In [0]:
lst = []
for n, s in zip (y.gi_smooth_3m, y.month_4): #make new column Apr with data
  lst.append(n*s) 
y['Apr'] = lst

In [0]:
lst = []
for n, s in zip (y.gi_smooth_3m, y.month_5): #make new column May with data
  lst.append(n*s) 
y['May'] = lst

In [0]:
lst = []
for n, s in zip (y.gi_smooth_3m, y.month_6): #make new column June with data
  lst.append(n*s) 
y['June'] = lst
y.head()

,cif_id,dlk_cob_date,gi_smooth_3m,cu_age,month,month_1,month_10,month_11,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,Jan,Feb,March,Apr,May,June
254464,1000,2018-01-31,53.184058,32.0,month_1,1,0,0,0,0,0,0,0,0,0,0,53.184058,0.000000,0.00000,0.000000,0.000000,0.0
254461,1000,2018-02-28,47.592137,33.0,month_2,0,0,0,1,0,0,0,0,0,0,0,0.000000,47.592137,0.00000,0.000000,0.000000,0.0
254460,1000,2018-03-31,40.828960,33.0,month_3,0,0,0,0,1,0,0,0,0,0,0,0.000000,0.000000,40.82896,0.000000,0.000000,0.0
254462,1000,2018-04-30,35.967168,33.0,month_4,0,0,0,0,0,1,0,0,0,0,0,0.000000,0.000000,0.00000,35.967168,0.000000,0.0
254465,1000,2018-05-31,61.794418,33.0,month_5,0,0,0,0,0,0,1,0,0,0,0,0.000000,0.000000,0.00000,0.000000,61.794418,0.0


In [0]:
def my_sum(x_df):
  dfout = sum (x_df['Jan'])
  return dfout
y_Jan = y.groupby(['cif_id']).apply(my_sum)
y_Jan.head(13)

cif_id
1000    5.318406e+01
1003    6.905228e+01
1006    4.956029e+00
1008   -8.000000e-07
1012    2.809535e+00
1015    7.121804e+01
1017    8.143111e+00
1026    4.369304e+01
1028    1.347130e+01
1040    1.895247e+02
1042    8.196364e+00
1044    0.000000e+00
1054    5.769253e+00
dtype: float64

In [0]:
def my_sum(x_df):
  dfout = sum (x_df['Feb'])
  return dfout
y_Feb = y.groupby(['cif_id']).apply(my_sum)
y_Feb.shape

(113962,)

In [0]:
def my_sum(x_df):
  dfout = sum (x_df['March'])
  return dfout
y_March = y.groupby(['cif_id']).apply(my_sum)
y_March.shape

(113962,)

In [0]:
def my_sum(x_df):
  dfout = sum (x_df['Apr'])
  return dfout
y_Apr = y.groupby(['cif_id']).apply(my_sum)
y_Apr.shape

(113962,)

In [0]:
def my_sum(x_df):
  dfout = sum (x_df['May'])
  return dfout
y_May = y.groupby(['cif_id']).apply(my_sum)
y_May.shape

(113962,)

In [0]:
def my_sum(x_df):
  dfout = sum (x_df['June'])
  return dfout
y_June = y.groupby(['cif_id']).apply(my_sum)
y_June.shape

(113962,)

In [0]:
#X_features = ['Jan', 'Feb', 'March', 'Apr', 'May', 'June']
y = pd.concat([y_Jan, y_Feb, y_March, y_Apr, y_May, y_June], axis=1)
y.head(13)

,0,1,2,3,4,5
cif_id,,,,,,
1000,5.318406e+01,47.592137,40.828960,35.967168,61.794418,84.276016
1003,6.905228e+01,70.795953,63.202465,83.535120,85.350595,78.765967
1006,4.956029e+00,4.040423,3.129146,1.719312,0.753373,0.000135
1008,-8.000000e-07,0.000009,0.000009,2.689611,2.693220,2.699090
1012,2.809535e+00,10.821343,11.756494,11.902506,4.437911,2.929028
1015,7.121804e+01,68.498085,66.233882,63.915932,65.449802,76.350342
1017,8.143111e+00,9.709545,11.011287,7.612845,9.155981,9.355318
1026,4.369304e+01,42.390199,41.170493,46.691882,116.524133,145.113503
1028,1.347130e+01,10.368094,2.750781,1.482965,1.000775,0.487881


In [0]:
y.columns

RangeIndex(start=0, stop=6, step=1)

In [0]:
X_features = range(0, 6, 1)
X_test = y[X_features]
X_test.head()

,0,1,2,3,4,5
cif_id,,,,,,
1000,5.318406e+01,47.592137,40.828960,35.967168,61.794418,84.276016
1003,6.905228e+01,70.795953,63.202465,83.535120,85.350595,78.765967
1006,4.956029e+00,4.040423,3.129146,1.719312,0.753373,0.000135
1008,-8.000000e-07,0.000009,0.000009,2.689611,2.693220,2.699090
1012,2.809535e+00,10.821343,11.756494,11.902506,4.437911,2.929028


In [0]:
prediction = net.predict(X_test)
print (prediction)

[[ 98.4441945  101.53781121  95.64676666  92.46501758  90.9128614
   87.83545449]
 [ 66.15862943  58.08543563  61.78425114  61.14995385  59.71734417
   58.33172327]
 [ -0.4161174    1.05203472   0.9611759    2.1533314    3.08551702
    3.80276519]
 ...
 [  1.26664634   2.226062     2.57405926   3.62714726   4.5290154
    5.15156403]
 [ 98.33320941  94.8560512   95.2832675   92.41349354  89.7321803
   86.54034096]
 [ 87.83593445  92.59875025  88.34660033  85.06148217  83.00719723
   79.78783127]]


In [0]:
df_pred = pd.DataFrame(prediction)
df_pred.columns = [ "July", "Aug", "Sep", "Oct", "Nov", "Dec"]
df_pred.head()

,July,Aug,Sep,Oct,Nov,Dec
0,98.444195,101.537811,95.646767,92.465018,90.912861,87.835454
1,66.158629,58.085436,61.784251,61.149954,59.717344,58.331723
2,-0.416117,1.052035,0.961176,2.153331,3.085517,3.802765
3,2.924494,3.287147,3.979457,4.695482,5.655820,6.119005
4,2.483375,3.778765,4.479052,5.439839,6.006204,6.467521


In [0]:
df_pred.shape

(113962, 6)

In [0]:
for index, row in y.iterrows():
  print (index, row[0]+row[1]+row[2]+row[3]+row[4]+row[5])

Streaming output truncated to the last 5000 lines.
489522 242.0933472
489525 23.643908
489529 213.43751693072
489531 29.406196000000005
489533 651.7890057536001
489537 20.288486784240003
489543 59.13846647199999
489561 267.21575771848006
489562 2.6635669576
489572 32.653994399999995
489576 107.517098644864
489580 66.30590662152
489582 53.16408870734401
489590 1.3672288
489592 108.40854000000002
489599 28.326326400000003
489604 404.6182765518401
489617 6.706161599999999
489619 180.58499440000003
489621 217.42132437776004
489623 667.8629935211839
489624 348.08997127104004
489629 9.757373774728002
489633 37.0807288
489634 73.51618640000001
489635 6.55272529136
489636 33.900640800000005
489653 138.6912456
489656 213.2217551216
489657 83.23207760000001
489663 0.00039359999999999997
489668 295.40729360000006
489685 43.6829615881784
489689 342.19326857762405
489693 0.0090816
489694 4.988312
489697 244.1967136
489701 339.83227120000004
489704 967.3740050757216
489707 1140.3552138144
489714 126

In [0]:
for i, r in df_pred.iterrows():
  print (i, r)

Streaming output truncated to the last 5000 lines.
Name: 113247, dtype: float64
113248 0    10.519452
1    11.948026
2    13.361061
3    13.763968
4    14.148331
5    14.157826
Name: 113248, dtype: float64
113249 0    81.431068
1    81.881206
2    80.242669
3    77.792879
4    75.926412
5    73.282907
Name: 113249, dtype: float64
113250 0     3.632512
1     9.326784
2    15.630399
3    18.685836
4    19.932419
5    21.058753
Name: 113250, dtype: float64
113251 0    2.714675
1    4.210828
2    4.995681
3    5.851729
4    6.538582
5    6.961591
Name: 113251, dtype: float64
113252 0    161.479121
1    162.214165
2    159.860737
3    154.506178
4    149.054912
5    143.161020
Name: 113252, dtype: float64
113253 0    48.489930
1    49.307398
2    48.747171
3    47.558658
4    46.770917
5    45.356203
Name: 113253, dtype: float64
113254 0    44.454952
1    41.662292
2    43.205417
3    42.652232
4    41.990337
5    41.005934
Name: 113254, dtype: float64
113255 0    75.192382
1    82.877274
2

In [0]:
y['6m'] = 0

In [0]:
y.head()

,0,1,2,3,4,5,6m
cif_id,,,,,,,
1000,5.318406e+01,47.592137,40.828960,35.967168,61.794418,84.276016,0
1003,6.905228e+01,70.795953,63.202465,83.535120,85.350595,78.765967,0
1006,4.956029e+00,4.040423,3.129146,1.719312,0.753373,0.000135,0
1008,-8.000000e-07,0.000009,0.000009,2.689611,2.693220,2.699090,0
1012,2.809535e+00,10.821343,11.756494,11.902506,4.437911,2.929028,0


In [0]:

lstInd = []

for index, row in y.iterrows():
  lstInd.append (index)
print(lstInd)


[1000, 1003, 1006, 1008, 1012, 1015, 1017, 1026, 1028, 1040, 1042, 1044, 1054, 1057, 1059, 1061, 1072, 1075, 1082, 1083, 1086, 1090, 1097, 1100, 1105, 1118, 1120, 1127, 1132, 1136, 1140, 1150, 1153, 1155, 1163, 1168, 1172, 1183, 1184, 1192, 1193, 1194, 1200, 1204, 1205, 1208, 1216, 1220, 1221, 1225, 1230, 1234, 1242, 1243, 1244, 1249, 1253, 1256, 1259, 1261, 1262, 1267, 1272, 1273, 1276, 1277, 1282, 1285, 1291, 1308, 1313, 1315, 1316, 1326, 1327, 1329, 1332, 1337, 1339, 1340, 1345, 1352, 1357, 1360, 1364, 1367, 1374, 1375, 1379, 1382, 1383, 1394, 1395, 1404, 1405, 1413, 1415, 1425, 1427, 1428, 1429, 1431, 1434, 1436, 1438, 1440, 1448, 1451, 1455, 1456, 1460, 1461, 1467, 1470, 1471, 1478, 1483, 1484, 1492, 1495, 1503, 1512, 1516, 1517, 1523, 1524, 1527, 1547, 1549, 1555, 1556, 1557, 1558, 1560, 1568, 1570, 1575, 1582, 1583, 1588, 1591, 1606, 1612, 1616, 1619, 1622, 1628, 1641, 1644, 1647, 1648, 1650, 1652, 1653, 1654, 1657, 1661, 1665, 1667, 1683, 1694, 1699, 1702, 1711, 1713, 1715, 171

In [0]:
lstValue = []

for index, row in df_pred.iterrows():
  lstValue.append (row['July']+row['Aug']+row['Sep']+row['Oct']+row['Nov']+row['Dec'])
print(lstValue)

[566.8421058296697, 365.2273375044262, 10.63870683017273, 26.66140457668152, 28.65475546185241, 497.66983637275433, 63.807060291284245, 792.7653914008815, 11.560883305558855, 828.561545006048, 41.09560182116783, 466.8853370892286, 23.690973590188666, 408.7183298733196, 225.2163393026606, 731.222659135689, 160.55107829906254, 88.25878181598874, 399.4539948243438, 554.7294447215534, 249.02241775810336, 186.8462907796786, 170.14478198229412, 706.6189632134553, 88.2667032495985, 164.26104812229653, 525.7096936722791, 283.580579568655, 83.4447547373003, 55.96278350021203, 98.81946819243319, 299.4807772824374, 87.94243535742021, 16.132023191295524, 297.0319898980525, 176.9470424104666, 355.6715809849534, 10.946907307552062, 157.44594030200085, 676.1152108730248, 315.20273551765155, 22.48107863979845, 756.0576053778273, 826.9399503645328, 20.435968576584294, 103.24553727762788, 635.4192633458284, 39.89209853993471, 477.18415033628884, 95.82850654834223, 50.21937950927355, 88.38069690662651, 2

In [0]:
df_out = pd.DataFrame(lstValue, lstInd)

df_out.head()

,0
1000,566.842106
1003,365.227338
1006,10.638707
1008,26.661405
1012,28.654755


In [0]:
df_out.to_csv("CUP_IT_predictions.csv", header=False, mode='w', index=True)

In [0]:
from google.colab import files
files.download('CUP_IT_predictions.csv')